<a href="https://colab.research.google.com/github/FabianDus1002/Data-Mining-Team9/blob/model%2Ffinetuned-BERT/FineTuneSentimentBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install evaluate
!pip install transformers[torch]
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-many

In [ ]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split

In [ ]:
# Generate dataset from excel file

drive.mount('/content/drive')

train_data = '/content/drive/MyDrive/Colab Notebooks/FineTuneBERT/data/subset_studium_german.xlsx'

df = pd.read_excel(train_data)

Mounted at /content/drive


## Baseline Model

Fine-tune BERT with a simple hold-out method

In [ ]:
df.head()

# drop entries where null value
df = df.dropna()

# set column of label in first position
df = df.reindex(columns=["Studium_Label", "Studium"])

# subsitute:
# negative -> 0
# neutral -> 1
# positive -> 2

for index, row, in df.iterrows():
  if row["Studium_Label"] == "negative":
    df.at[index,'Studium_Label'] = 0
  elif row["Studium_Label"] == "neutral":
    df.at[index, "Studium_Label"] = 1
  elif row["Studium_Label"] == "positive":
    df.at[index, "Studium_Label"] = 2


# change column names
df["label"] = df["Studium_Label"]
df["text"] = df["Studium"]
df.drop(columns=["Studium", "Studium_Label"], inplace=True)

df_train, df_test = train_test_split(df, train_size=0.8)

df_train.reset_index(inplace=True)
df_test.reset_index(inplace=True)

In [ ]:
datasets = Dataset.from_pandas(df).train_test_split(test_size=0.2)

df_train = datasets["train"]
df_test = datasets["test"]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-cased")
def tokenize_function(data):
  return tokenizer(data["text"], padding="max_length", truncation=True)

tokenized_datasets = datasets.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/456 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

Map:   0%|          | 0/293 [00:00<?, ? examples/s]

Map:   0%|          | 0/74 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("dbmdz/bert-base-german-cased", num_labels=3)

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
metric = evaluate.load("f1")

NameError: name 'evaluate' is not defined

In [ ]:
def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels, average="micro")

In [ ]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    num_train_epochs=3,
    learning_rate=5e-05)

In [ ]:
trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=tokenized_datasets["train"],

    eval_dataset=tokenized_datasets["test"],

    compute_metrics=compute_metrics,
)

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 293
    })
    test: Dataset({
        features: ['label', 'text', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 74
    })
})

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [ ]:
trainer.get_num_trainable_parameters()

Training for following parameter spaces:

- Epoch: 3,4,5
- Learning Rate
- Dataset: Unbalanced, Balanced
- Size of training set: 300, 500, 700

## Different Epochs

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=3)

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=tokenized_datasets["train"],

    eval_dataset=tokenized_datasets["test"],

    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=4)

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=tokenized_datasets["train"],

    eval_dataset=tokenized_datasets["test"],

    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=5)

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=tokenized_datasets["train"],

    eval_dataset=tokenized_datasets["test"],

    compute_metrics=compute_metrics,
)

trainer.train()

## Build balanced training data set

The dataset is strongly imbalanced because the majority of responses are either neutral or positive. Only a small subset contains negative results. To have a well generalizing model, we need to ensure a balanced training data set. Therefore we use the approach of semi-supervised learning:

https://www.ibm.com/topics/semi-supervised-learning

With the approach of semi-supervised learning we were able to find a handful of other examples in the around 6000 records that we can additionally label as negative sentiment and use for our training purpose.

However, this process also needs a lot of manual effort because the sentiment assigned by BERT is in most cases not according to our understanding of a negative sentiment and hence need manual evaluation.

All training records that we have labeled as negative with this approach have the comment "semi-supervised" in the comment section.


In [ ]:
train_data = '/content/drive/MyDrive/Colab Notebooks/FineTuneBERT/data/subset_studium_german.xlsx'

df = pd.read_excel(train_data)

# drop entries that are already labeled
df = df[df["Studium_Label"].isna()]

# drop entries where text value is null
df = df.dropna(subset=["Studium"])

# set column of label in first position
df = df.reindex(columns=["Studium_Label", "Studium", "Unnamed: 0"])

# change column names
df["label"] = df["Studium_Label"]
df["text"] = df["Studium"]
df.drop(columns=["Studium", "Studium_Label"], inplace=True)

print(df.info())

unlabeled_dataset = Dataset.from_pandas(df)

tokenized_unlabeled_dataset = unlabeled_dataset.map(tokenize_function, batched=True)

print(tokenized_unlabeled_dataset)

<class 'pandas.core.frame.DataFrame'>
Index: 5102 entries, 9 to 6117
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5102 non-null   int64 
 1   label       0 non-null      object
 2   text        5102 non-null   object
dtypes: int64(1), object(2)
memory usage: 159.4+ KB
None


Map:   0%|          | 0/5102 [00:00<?, ? examples/s]

Dataset({
    features: ['Unnamed: 0', 'label', 'text', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5102
})


In [ ]:
predictions = trainer.predict(test_dataset=tokenized_unlabeled_dataset)

In [ ]:
index = 0
threshold = 0.75
for pred in predictions.predictions:
  if pred[0] > pred[1] and pred[0] > pred[2] and pred[0] > threshold:
    print (str(index) + ": negative, " + str(pred[0]))
    print("Dataframe Index in subset: " + str(tokenized_unlabeled_dataset[index]["Unnamed: 0"]))
    print(tokenized_unlabeled_dataset[index]["text"] + "\n")
  elif pred[1] > pred[0] and pred[1] > pred[2]:
    pass
    # print (str(index) + ": neutral, " + str(pred[0]))
  elif pred[2] > pred[0] and pred[2] > pred[1]:
     pass
     # print (str(index) + ": positive, " + str(pred[0]))
     # (tokenized_unlabeled_dataset[index]["text"] + "\n")

  index += 1

68: negative, 0.8579676
Dataframe Index in subset: 1289
In der ersten IBEA Generation waren die Semesterzeiten noch nicht an die generellen Zeiten der ESSEC angepasst wodurch das Kursangebot im ersten Jahr begrenzt war. Dies soll zumindest ab dem zweiten Jahr (FSS 2019) anders sein, was ich jedoch zu jetzigem Zeitpunkt nicht beurteilen kann. Grundsätzlich gibt es Anwesenheitspflicht und man darf nur 3 mal während des Semesters fehlen in einem Kurs. Die einzelnen Lectures sind mit knapp 3 Stunden deutlich länger als z.B. in Mannheim was je nach Kurs zu mehr oder weniger Qualität führt. In unregelmäßigen Abständen muss man Essays o.Ä. einreichen und hat dadurch relativ kontinuierlichen Arbeitsaufwand. Grundsätzlich ist dieser jedoch deutlich geringer als in Mannheim (vor allem zum Ende des Semesters).

73: negative, 0.8252365
Dataframe Index in subset: 1295
Das Studium ist, ähnlich wie die Organisation an der NEOMA, mittelmäßig bis extrem chaotisch.
Man belegt insgesamt maximal sechs Kur